In [7]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pathlib import Path
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.utils.data as data

from dataset import ImageDataset
import networks
from utils import train, epoch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model 1: All Classes

In [ ]:
hres_size = (512, 512)
scale_factor = 4

train_paths = []
for dir in Path('train').glob('**/*'):
    train_paths.extend(dir.glob('*.png'))

test_paths = []
for dir in Path('test').glob('**/*'):
    test_paths.extend(dir.glob('*.png'))

train_set = ImageDataset(train_paths, hres_size, scale_factor, None)
test_set = ImageDataset(test_paths, hres_size, scale_factor, None)

In [9]:
batch_size = 16

trainloader = data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
testloader = data.DataLoader(test_set, batch_size=1, shuffle=False)

In [ ]:
class Args():
    device = "cuda"
    lr_net = 0.005
    epoch_eval_train = 20
    save_path = Path(f"results/all")

args = Args()

In [5]:
net = networks.SRCNN(3, scale_factor)
train(0, net, trainloader, testloader, args)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
[2024-12-05 14:03:13] Evaluate_00: epoch = 0020 train time = 1163 s train loss = 0.002096 test loss = 0.002120


SRCNN(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU(inplace=True)
)

In [10]:
optimizer = torch.optim.SGD(net.parameters(), lr=float(args.lr_net), momentum=0.9, weight_decay=0.0005)
criterion = nn.MSELoss().to(args.device)
epoch('test', testloader, net, optimizer, criterion, args)

0.0021199920357993017

# Model 2: Only Young

In [ ]:
hres_size = (512, 512)
scale_factor = 4

train_paths = []
for dir in Path('train').glob('**/Young*'):
    train_paths.extend(dir.glob('*.png'))

test_paths = []
for dir in Path('test').glob('**/*'):
    test_paths.extend(dir.glob('*.png'))

train_set = ImageDataset(train_paths, hres_size, scale_factor, None)
test_set = ImageDataset(test_paths, hres_size, scale_factor, None)

In [27]:
batch_size = 16

trainloader = data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
testloader = data.DataLoader(test_set, batch_size=1, shuffle=False)

In [ ]:
class Args():
    device = "cuda"
    lr_net = 0.005
    epoch_eval_train = 20
    save_path = Path(f"results/half")

args = Args()

In [29]:
net = networks.SRCNN(3, scale_factor)
train(0, net, trainloader, testloader, args)

Epoch 0
Epoch 1
[2024-12-05 15:01:51] Evaluate_00: epoch = 0001 train time = 55 s train loss = 0.007349 test loss = 0.006475


SRCNN(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU(inplace=True)
)

# Model 3: Young + 1 Old of each class repeated

In [35]:
hres_size = (512, 512)
scale_factor = 4

train_paths = []
for dir in Path('train').glob('**/Young*'):
    train_paths.extend(dir.glob('*.png'))
class_length = len(train_paths) // 2
for dir in Path('train').glob('**/Old*'):
    train_paths.extend([list(dir.glob('*.png'))[0]] * class_length)

test_paths = []
for dir in Path('test').glob('**/*'):
    test_paths.extend(dir.glob('*.png'))

train_set = ImageDataset(train_paths, hres_size, scale_factor, None)
test_set = ImageDataset(test_paths, hres_size, scale_factor, None)

In [36]:
batch_size = 16

trainloader = data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
testloader = data.DataLoader(test_set, batch_size=1, shuffle=False)

In [37]:
class Args():
    device = "cuda"
    lr_net = 0.005
    epoch_eval_train = 20
    save_path = Path("results/half+repetition")

args = Args()

In [38]:
net = networks.SRCNN(3, scale_factor)
train(0, net, trainloader, testloader, args)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
[2024-12-05 15:39:00] Evaluate_00: epoch = 0020 train time = 1166 s train loss = 0.001889 test loss = 0.002295


SRCNN(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU(inplace=True)
)

# Predicting

In [42]:
image_dir = Path('results/all')

image_paths = []
for dir in image_dir.glob('**/*'):
    image_paths.extend(dir.glob('*.png'))

with open(image_dir / "images.csv", 'w') as f:
    f.write("img_path\n")
    for image_path in image_paths:
        f.write(f"{image_path}\n")

In [44]:
image_dir = Path('results/half')

image_paths = []
for dir in image_dir.glob('**/*'):
    image_paths.extend(dir.glob('*.png'))

with open(image_dir / "images.csv", 'w') as f:
    f.write("img_path\n")
    for image_path in image_paths:
        f.write(f"{image_path}\n")

In [45]:
image_dir = Path('results/half+repetition')

image_paths = []
for dir in image_dir.glob('**/*'):
    image_paths.extend(dir.glob('*.png'))

with open(image_dir / "images.csv", 'w') as f:
    f.write("img_path\n")
    for image_path in image_paths:
        f.write(f"{image_path}\n")